# Operating Room Scheduling



In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
import pandas as pd

#### Part (a): Minimize the total under-allocation of each department to operating rooms

##### Parameters

In [2]:
#                     i      target share (t)
# General Surgery |   1          0.484  
# Emergency       |   2          0.042
# Neurosurgery    |   3          0.253
# Opthalmology    |   4          0.074
# Oral Surgery    |   5          0.053
# Otolaryngology  |   6          0.095

# Target times for the 6 departments as per the given proportion that has to be consistent
t_i = [103.334, 8.967, 54.0155, 15.799, 11.3155, 20.2825]

# Given number of hours for each day and operating room
hours = [[9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 8, 8, 8, 6.5]]

##### Defining the model, decision variables

In [3]:
m1 = Model()

# Number of departments, rooms, and days
d = 6
r = 5
days = 5

# x1 stores the binary decision variable (x[ijk]) which indicates 1 if the given room/day is booked and 0 if it is not
x1 = m1.addVars(d,r,days,lb=0,vtype = GRB.BINARY)

# h_a stores the actual time allocations
h_a = m1.addVars(d)

# under_a stores the underallocations
under_a = m1.addVars(d)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05


##### Constraints

In [4]:
#  Constraint to compute actual hours assigned to each department
for i in range(d):
    m1.addConstr(h_a[i] == sum(x1[i, j, k] * hours[j][k] for j in range(r) for k in range(days)))
    
# Constraint to compute underallocations based on hours allocated to each department
for i in range(d):
    m1.addConstr(under_a[i] >= t_i[i] - h_a[i])
    
# Constraint to ignore overallocations in the model
for i in range(d):
    m1.addConstr(under_a[i] >= 0.0)
    
# Constraint to make sure only one department is booked to a given room on a given day
for j in range(r):
    for k in range(days):
        m1.addConstr(sum(x1[i, j, k] for i in range(d)) <= 1)

##### Objective Function

In [5]:
m1.setObjective(sum(under_a[i] / t_i[i] for i in range(d)), GRB.MINIMIZE)

##### Update and run the model

In [6]:
m1.update()
m1.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43 rows, 162 columns and 324 nonzeros
Model fingerprint: 0x7864d1b7
Variable types: 12 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 12 rows and 6 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 2.066116e-03, 57 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

### We find that the underallocation percentage is 5.19%

#### Part (b): Incorporating constraints into your base model to ensure that no department is allocated rooms on two different floors on the same day

##### Additional parameter

In [7]:
# Operating rooms in each floor
floors = {0:[0,1],1:[2,3],2:[4]}

##### Defining the model, decision variables

In [8]:
m2 = Model()

# x1 stores the binary decision variable (x[ijk]) which indicates 1 if the given room/day is booked and 0 if it is not
x2 = m2.addVars(d,r,days,lb=0,vtype = GRB.BINARY)

# h_b stores the actual time allocations
h_b = m2.addVars(d)

# under_b stores the underallocations
under_b = m2.addVars(d)

# z stores the allocations by floors (3)
z = m2.addVars(d,days,3,lb=0,vtype=GRB.INTEGER)

##### Constraints

In [9]:
#  Constraint to compute actual hours assigned to each department
for i in range(d):
    m2.addConstr(h_b[i] == sum(x2[i, j, k] * hours[j][k] for j in range(r) for k in range(days)))

# For each department, day and floor, the allocations by floor:
for i in range(d):
    for j in range(days):
        for l in range(3):
            m2.addConstr(z[i,j,l] == sum(x2[i, j, fl] for fl in floors[l]))

In [10]:
# y is a binary decision variable that stores 1 if a floor has an allocation for given dept and day
y = m2.addVars(d,days,3,vtype = GRB.BINARY)

# M is assigned a large positive number
M = 20

for i in range(d):
    for j in range(days):
        
        m2.addConstr(z[i,j,0] <= M*y[i,j,0])
        m2.addConstr(z[i,j,1] <= M*y[i,j,1])
        m2.addConstr(z[i,j,2] <= M*y[i,j,2])
        
        m2.addConstr(z[i,j,0] >= y[i,j,0])
        m2.addConstr(z[i,j,1] >= y[i,j,1])
        m2.addConstr(z[i,j,2] >= y[i,j,2])
        
        # Enforces that only one floor is used by each department in a day
        m2.addConstr(y[i,j,0]+y[i,j,1]+y[i,j,2]<=1)

In [11]:
# Constraint to compute underallocations based on hours allocated to each department
for i in range(d):
    m2.addConstr(under_b[i] >= t_i[i] - h_b[i])
    
# Constraint to ignore overallocations in the model
for i in range(d):
    m2.addConstr(under_b[i] >= 0.0)
    
# Constraint to make sure only one department is booked to a given room on a given day
for j in range(r):
    for k in range(days):
        m2.addConstr(sum(x2[i, j, k] for i in range(d)) <= 1)

##### Objective Function

In [12]:
m2.setObjective(sum(under_b[i] / t_i[i] for i in range(d)), GRB.MINIMIZE)

##### Update and run the model

In [13]:
m2.update()
m2.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 343 rows, 342 columns and 1014 nonzeros
Model fingerprint: 0x4c4e195b
Variable types: 12 continuous, 330 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 162 rows and 126 columns
Presolve time: 0.00s
Presolved: 181 rows, 216 columns, 756 nonzeros
Variable types: 4 continuous, 212 integer (210 binary)
Found heuristic solution: objective 5.9998574

Root relaxation: objective 1.387152e-01, 119 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

#### Optimal value after minimization is 0.138